In [1]:
import re
import os
from langchain.vectorstores.neo4j_vector import Neo4jVector
from langchain.document_loaders import WikipediaLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import (
    CharacterTextSplitter,
    RecursiveCharacterTextSplitter,
)
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from langchain.text_splitter import TokenTextSplitter

from secret import *
import chromadb

os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [2]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
raw_documents = WikipediaLoader(query="Atelectasis").load()
text_splitter = TokenTextSplitter(chunk_size=2048, chunk_overlap=24)
documents = text_splitter.split_documents([raw_documents[0]]) # should I only take the first? which is the most relevant?

c:\Users\mike8\anaconda3\envs\pytorch\lib\site-packages\wikipedia\wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html.parser"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file c:\Users\mike8\anaconda3\envs\pytorch\lib\site-packages\wikipedia\wikipedia.py. To get rid of this warning, pass the additional argument 'features="html.parser"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


In [3]:
from langchain.indexes import GraphIndexCreator
from langchain.graphs.networkx_graph import KnowledgeTriple
from langchain.llms import OpenAI

In [4]:
index_creator = GraphIndexCreator(llm=OpenAI(temperature=0))

c:\Users\mike8\anaconda3\envs\pytorch\lib\site-packages\langchain_core\_api\deprecation.py:115: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use langchain_openai.OpenAI instead.
  warn_deprecated(


In [5]:
graph = None
for d in documents:
    g_temp = index_creator.from_text(d.page_content)
    if graph is None:
        graph = g_temp
    else:
        triplets = graph.get_triples()
        for t in triplets:
            graph.add_triple(knowledge_triple=KnowledgeTriple(*t))

In [6]:
from langchain.chains import GraphQAChain

In [7]:
chain = GraphQAChain.from_llm(OpenAI(temperature=0), graph=graph, verbose=True)

In [8]:
lesion = "Atelectasis"
questions = [
    f"What is {lesion}?",
    f"What are the symptoms associated with {lesion}?",
    f"What can cause {lesion}?",
    f"What are the patient’s symptoms that are relevant for {lesion}?",
    f"What are the relevant clinical signs for the etiological diagnosis of {lesion}?",
    f"What are the relevant laboratory data for the etiological diagnosis of {lesion}?",
    f"What are the relevant clinical characteristics for the etiological diagnosis of {lesion}",
    f"What are the patient’s personal relevant history for the etiological diagnosis of {lesion}",
]

In [9]:
res_dict = {}
for q in questions:
    res = chain.run(q)
    res_dict[q] = res.strip()

c:\Users\mike8\anaconda3\envs\pytorch\lib\site-packages\langchain_core\_api\deprecation.py:115: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new GraphQAChain chain...
Entities Extracted:
 Atelectasis
Full Context:
Atelectasis is the partial collapse or closure of a lung resulting in reduced or absent gas exchange
Atelectasis is usually unilateral
Atelectasis affects part or all of one lung
Atelectasis is a condition
Atelectasis is where the alveoli are deflated down to little or no volume
Atelectasis is distinct from pulmonary consolidation
Atelectasis is often referred to informally as a collapsed lung
Atelectasis is a very common finding in chest X-rays and other radiological studies
Atelectasis may be caused by normal exhalation
Atelectasis may be caused by various medical conditions
Atelectasis is not synonymous with a pneumothorax
Atelectasis may occur as a post-operative complication
Atelectasis may occur as a result of surfactant deficiency
Atelectasis leads to infant respiratory distress syndrome
Atelectasis uses combining form

> Finished chain.


> Entering new GraphQAChain chain...
Entities Extracted

In [10]:
for q in questions:
    print(res_dict[q])

Atelectasis is a condition where part or all of one lung is partially collapsed or closed, resulting in reduced or absent gas exchange. It is often referred to informally as a collapsed lung and is a very common finding in chest X-rays and other radiological studies. It can be caused by normal exhalation, various medical conditions, or as a post-operative complication. It is distinct from a pneumothorax and may occur as a result of surfactant deficiency, leading to infant respiratory distress syndrome. Atelectasis is also a combining form used in medical terminology.
Reduced or absent gas exchange, unilateral collapse or closure of a lung, deflated alveoli, and infant respiratory distress syndrome.
Various medical conditions, normal exhalation, post-operative complication, and surfactant deficiency can all cause Atelectasis.
Reduced or absent gas exchange, unilateral lung collapse, deflated alveoli, and respiratory distress syndrome.
Reduced or absent gas exchange, unilateral presentat

In [11]:
raise StopIteration()

StopIteration: 

In [ ]:
chain.run("What is Atelectasis?")



> Entering new GraphQAChain chain...
Entities Extracted:
 Atelectasis
Full Context:
Atelectasis is the collapse or closure of a lung
Atelectasis is usually unilateral
Atelectasis is a condition where the alveoli are deflated down to little or no volume
Atelectasis is often called a collapsed lung
Atelectasis is not synonymous with a pneumothorax
Atelectasis may occur as a post-operative complication or as a result of surfactant deficiency
Atelectasis leads to infant respiratory distress syndrome
Atelectasis is caused by normal exhalation
Atelectasis is caused by various medical conditions
Atelectasis is associated with post-op fever

> Finished chain.


' Atelectasis is the collapse or closure of a lung, where the alveoli are deflated down to little or no volume. It is often called a collapsed lung and is not synonymous with a pneumothorax. It may occur as a post-operative complication or as a result of surfactant deficiency and can lead to infant respiratory distress syndrome. It is caused by normal exhalation and various medical conditions and is associated with post-op fever.'

In [ ]:
chain.run("What are the symptoms of Atelectasis?")



> Entering new GraphQAChain chain...
Entities Extracted:
 Atelectasis
Full Context:
Atelectasis is the collapse or closure of a lung
Atelectasis is usually unilateral
Atelectasis is a condition where the alveoli are deflated down to little or no volume
Atelectasis is often called a collapsed lung
Atelectasis is not synonymous with a pneumothorax
Atelectasis may occur as a post-operative complication or as a result of surfactant deficiency
Atelectasis leads to infant respiratory distress syndrome
Atelectasis is caused by normal exhalation
Atelectasis is caused by various medical conditions
Atelectasis is associated with post-op fever

> Finished chain.


' Symptoms of Atelectasis include shortness of breath, chest pain, coughing, and difficulty breathing.'

In [ ]:
chain.run("What can cause Atelectasis?")



> Entering new GraphQAChain chain...
Entities Extracted:
 Atelectasis
Full Context:
Atelectasis is the collapse or closure of a lung
Atelectasis is usually unilateral
Atelectasis is a condition where the alveoli are deflated down to little or no volume
Atelectasis is often called a collapsed lung
Atelectasis is not synonymous with a pneumothorax
Atelectasis may occur as a post-operative complication or as a result of surfactant deficiency
Atelectasis leads to infant respiratory distress syndrome
Atelectasis is caused by normal exhalation
Atelectasis is caused by various medical conditions
Atelectasis is associated with post-op fever

> Finished chain.


' Atelectasis can be caused by normal exhalation, various medical conditions, and surfactant deficiency. It can also occur as a post-operative complication or as a result of infant respiratory distress syndrome.'

In [ ]:
chain.run("What clinical features radiologists need for diagnosing Atelectasis?")



> Entering new GraphQAChain chain...
Entities Extracted:
 Atelectasis
Full Context:
Atelectasis is the collapse or closure of a lung
Atelectasis is usually unilateral
Atelectasis is a condition where the alveoli are deflated down to little or no volume
Atelectasis is often called a collapsed lung
Atelectasis is not synonymous with a pneumothorax
Atelectasis may occur as a post-operative complication or as a result of surfactant deficiency
Atelectasis leads to infant respiratory distress syndrome
Atelectasis is caused by normal exhalation
Atelectasis is caused by various medical conditions
Atelectasis is associated with post-op fever

> Finished chain.


' Radiologists need to look for signs of unilateral collapse or closure of a lung, deflated alveoli, and post-operative fever. They may also need to consider other medical conditions that could be causing the Atelectasis.'

In [ ]:
chain.run("what are radiographic features of Atelectasis?")



> Entering new GraphQAChain chain...
Entities Extracted:
 Atelectasis
Full Context:
Atelectasis is the collapse or closure of a lung
Atelectasis is usually unilateral
Atelectasis is a condition where the alveoli are deflated down to little or no volume
Atelectasis is often called a collapsed lung
Atelectasis is not synonymous with a pneumothorax
Atelectasis may occur as a post-operative complication or as a result of surfactant deficiency
Atelectasis leads to infant respiratory distress syndrome
Atelectasis is caused by normal exhalation
Atelectasis is caused by various medical conditions
Atelectasis is associated with post-op fever

> Finished chain.


' Radiographic features of Atelectasis include a decrease in lung volume, a decrease in the size of the affected lung, and a decrease in the density of the affected lung.'